<a href="https://colab.research.google.com/github/kita-ge/colab/blob/main/%E6%B0%B4%E5%B9%B3%E6%96%AD%E9%9D%A2%EF%BC%86%E3%83%A2%E3%83%87%E3%83%AB%E5%88%87%E6%96%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#入力:[geometries]←List,geo
#     [ids]←List,Guid
#     [height]←Item,float
#出力:[a]
import Rhino.Geometry as rg
import Rhino

def create_cut_lines():
    tol = Rhino.RhinoDoc.ActiveDoc.ModelAbsoluteTolerance
    cut_plane = rg.Plane(rg.Point3d(0, 0, height), rg.Vector3d.ZAxis)
    cut_list = []

    input_geos = geometries if isinstance(geometries, list) else [geometries]

    for geo in input_geos:
        if not geo: continue

        if isinstance(geo, rg.Mesh):
            plines = rg.Intersect.Intersection.MeshPlane(geo, cut_plane)
            if plines:
                for p in plines:
                    if p.Count > 0: cut_list.append(rg.PolylineCurve(p))

        else:
            brep = geo.ToBrep() if hasattr(geo, "ToBrep") else geo
            if isinstance(brep, rg.Brep):
                _, crvs, _ = rg.Intersect.Intersection.BrepPlane(brep, cut_plane, tol)
                if crvs: cut_list.extend(crvs)

    return rg.Curve.JoinCurves(cut_list, tol)

a = create_cut_lines()

In [ ]:
#入力:[geometries]←List,geo
#     [height]←Item,float
#出力:[a]
import Rhino.Geometry as rg
import Rhino

def get_geometries_below():
    tol = Rhino.RhinoDoc.ActiveDoc.ModelAbsoluteTolerance
    cut_plane = rg.Plane(rg.Point3d(0, 0, height), rg.Vector3d.ZAxis)

    output_geos = []
    input_geos = geometries if isinstance(geometries, list) else [geometries]

    for geo in input_geos:
        if geo is None: continue

        bbox = geo.GetBoundingBox(True)
        if bbox.Max.Z <= height:
            output_geos.append(geo)
            continue
        if bbox.Min.Z >= height:
            continue

        if isinstance(geo, rg.Brep) or hasattr(geo, "ToBrep"):
            brep = geo.ToBrep() if hasattr(geo, "ToBrep") else geo
            trimmed_brep = brep.Trim(cut_plane, tol)
            if trimmed_brep:
                output_geos.append(trimmed_brep)

        elif isinstance(geo, rg.Mesh):
            parts = geo.Split(cut_plane)
            if parts:
                for p in parts:
                    if p.GetBoundingBox(True).Center.Z < height:
                        output_geos.append(p)

    return output_geos

a = get_geometries_below()